In [ ]:
import numpy as np
from glob import glob

# load filenames for human and dog images
dog_files = np.array(glob("data/dog_images/*/*/*"))

# print number of images in each dataset
print('There are %d total dog images.' % len(dog_files))

In [ ]:
import os
import torch
from torchvision import datasets
from torchvision import transforms

train_transform = transforms.Compose([transforms.RandomResizedCrop(244),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomRotation(20),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])]) 
test_transform = transforms.Compose([transforms.Resize(225),
                                     transforms.CenterCrop(244),
                                    transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])

train_dset = datasets.ImageFolder('/data/dog_images/train/', transform=train_transform)
val_dset = datasets.ImageFolder('/data/dog_images/valid/', transform=test_transform)
test_dset = datasets.ImageFolder('/data/dog_images/test/', transform=test_transform)

loaders_scratch = {
    'train' : torch.utils.data.DataLoader(train_dset, batch_size=32, shuffle=True),
    'test' : torch.utils.data.DataLoader(test_dset, batch_size=32, shuffle=True),
    'valid' : torch.utils.data.DataLoader(val_dset, batch_size=32, shuffle=True)
}


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        ## layers of a CNN

        self.conv1 = nn.Conv2d(3,16,5,padding=2)
        self.conv2 = nn.Conv2d(16,32,3,padding=1)
        self.conv3 = nn.Conv2d(32,64,3,padding=1)
        self.conv4 = nn.Conv2d(64,128,3,padding=1)

        self.bc1 = nn.BatchNorm2d(16)
        self.bc2 = nn.BatchNorm2d(32)
        self.bc3 = nn.BatchNorm2d(64)
        self.bc4 = nn.BatchNorm2d(128)   
        
        self.pool = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(15*15*128,1024)
        self.fc2 = nn.Linear(1024,512)
        self.fc3 = nn.Linear(512,133)
        
        self.bc_fc1 = nn.BatchNorm1d(1024)
        self.bc_fc2 = nn.BatchNorm1d(512)
    
        self.dropout = nn.Dropout2d(p=.5)
    def forward(self, x):
        ## forward behavior
        x = self.bc1(self.pool(F.relu(self.conv1(x))))
        x = self.bc2(self.pool(F.relu(self.conv2(x))))        
        x = self.bc3(self.pool(F.relu(self.conv3(x))))
        x = self.bc4(self.pool(F.relu(self.conv4(x))))

        x = self.dropout(x.view(x.shape[0], -1))
        
        x = self.dropout(self.bc_fc1(F.relu(self.fc1(x))))
        x = self.dropout(self.bc_fc2(F.relu(self.fc2(x))))
        x = F.log_softmax(x, dim = 1)
  
        return x



# instantiate the CNN
model_scratch = Net()
print(model_scratch)

# # move tensors to GPU if CUDA is available
if True:
    model_scratch.cuda()

### Loss Function and Optimizer



In [ ]:
import torch.optim as optim

criterion_scratch = nn.NLLLoss()

optimizer_scratch = optim.Adam(model_scratch.parameters(), lr=0.001)


### Train and Validate the Model


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output,target)
            loss.backward()
            optimizer.step()
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
      
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)            
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))


        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        if valid_loss < valid_loss_min:
            print('Saving Model')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    return model


# train the model
model_scratch = train(50, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, True, 'model_scratch.pt')

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('saved_models/model_scratch.pt'))

### Test the Model


In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    
test(loaders_scratch, model_scratch, criterion_scratch, True)